# Criação de dim produto

## Imports

In [0]:
from pyspark.sql.functions import monotonically_increasing_id, col, when
from pyspark.sql.types import StringType

##Selecionando dados da tabela vendas da camada prata

In [0]:
df_silver_produto = spark.sql("""
                      SELECT DISTINCT
                         principio_ativo,
                         descricao_apresentacao,
                         unidade_medida,
                         CID10
                      FROM controlled_medication_data.silver_layer.vendas
                      """)

In [0]:
df_silver_produto.display()

In [0]:
def classificar_formato(descricao):
    if descricao:
        descricao = descricao.upper()

        if "MG CAP" in descricao or "CAP GEL" in descricao or "CAP DURA" in descricao:
            return "Cápsula"
        elif "COM REV" in descricao or "COM CT" in descricao or "COMPRIMIDO" in descricao:
            return "Comprimido"
        elif "MG/ML" in descricao or "ML SUS" in descricao or "ML" in descricao:
            return "ML"
        elif "MG/G" in descricao or "PO" in descricao:
            return "Grama"
    
    return "Outro" 


In [0]:
classificar_formato_udf = udf(classificar_formato, StringType())

In [0]:
df_com_formato = df_silver_produto.withColumn("formato", classificar_formato_udf(col("descricao_apresentacao")))

In [0]:
df_com_formato.display()

In [0]:
df_dim_produto = df_com_formato

##Criando id para cada linha da tabela dim (pk)

In [0]:
df_dim_produto = df_dim_produto.withColumn("id_produto", monotonically_increasing_id())

In [0]:
df_dim_produto.display()

##Salvando tabela

In [0]:
df_dim_produto.write.mode("overwrite") \
    .saveAsTable("controlled_medication_data.gold_layer.dim_produto")